In [1]:
import pandas as pd
from collections import Counter,defaultdict
import re
import jieba
import numpy as np

In [2]:
def get_ngram_model_dict(file_dir='movie_comments.csv'):
    sourcedata=pd.read_csv(file_dir,encoding='utf-8')
    two_gram_dict=defaultdict(int)
    one_gram_dict=defaultdict(int)
    for comment in sourcedata['comment'].dropna():
        comment=str(comment)
        if not comment.strip():continue
        tokened_comment=''.join(re.findall(r'\w+',comment))
        words=list(jieba.cut(tokened_comment))
        if not words:continue
        for index,word in enumerate(words[:-1]):
            one_gram_dict[word]+=1
            two_gram_dict[words[index]+words[index+1]]+=1
        one_gram_dict[words[-1]]+=1
    return one_gram_dict,two_gram_dict        

In [3]:
one_gram_dict,two_gram_dict=get_ngram_model_dict(file_dir='movie_comments.csv')

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (0,4) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Dumping model to file cache /tmp/jieba.cache
Dump cache file failed.
Traceback (most recent call last):
  File "/home/stu_18701958249/.local/lib/python3.7/site-packages/jieba/__init__.py", line 154, in initialize
    _replace_file(fpath, cache_file)
PermissionError: [Errno 1] Operation not permitted: '/tmp/tmpcaitim78' -> '/tmp/jieba.cache'
Loading model cost 0.896 seconds.
Prefix dict has been built successfully.


![ngram_formula1](./ngram_formula1.jpg)

![ngram_formula2](./ngram_formula2.jpg)

In [4]:
def get_sentence_probability(sentence:str,one_gram_dict:dict,two_gram_dict:dict):
    new_sentence=''.join(re.findall(r'\w+',sentence))
    words=list(jieba.cut(new_sentence))
    final_prob=1.0
    for index,word in enumerate(words[1:]):
        final_prob*=two_gram_dict.get(words[index-1]+words[index],1.0)*1.0/one_gram_dict.get(words[index-1],len(one_gram_dict))
    final_prob*=one_gram_dict.get(words[0],1.0)/sum(one_gram_dict.values())
    return final_prob    

In [5]:
sentence1='今天不好天气，我打算不出去'
print(get_sentence_probability(sentence1,one_gram_dict,two_gram_dict))
sentence2='今天天气不好，我不打算出去'
print(get_sentence_probability(sentence2,one_gram_dict,two_gram_dict))
sentence3='今天是个好天气'
print(get_sentence_probability(sentence3,one_gram_dict,two_gram_dict))

1.4531058008952092e-20
1.898952810001505e-16
5.951564476572636e-11
